# Demand side flexibility


In [1]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JuMP")
Pkg.add("Gurobi")
#Pkg.add("NLopt")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [2]:
using CSV, DataFrames, JuMP, Gurobi

# Read the CSV data

In [3]:
SCALE_MWH = 1000
SCALE_DOLLAR = 1000;

In [4]:
fixed_cost = CSV.read("fixed_cost_2.csv",DataFrame) ./ SCALE_DOLLAR

Row,Fixed Cost,Fixed OM cost
,Float64,Float64
1,543.5,21.7531
2,666.821,28.7795
3,1110.5,37.5
4,3415.0,103.5
5,434.537,0.0
6,186.085,23.3723


In [5]:
variable_cost = CSV.read("variable_cost_2.csv",DataFrame) ./ SCALE_DOLLAR

Row,Variable OM,Ramp,Start
,Float64,Float64,Float64
1,0.0,0.0,0.0
2,0.0,0.0,0.002
3,0.00589,0.006,0.035
4,0.011176,0.015,0.07
5,0.0,0.001,0.0
6,0.0,0.0,0.0


In [6]:
co2_intensity = CSV.read("co2_intensity.csv",DataFrame)

Row,CO2 Intensity
,Int64
1,0
2,0
3,452
4,965
5,0
6,0


In [7]:
P_max_s = 95897829.43 / SCALE_MWH
P_max_w = 11158008 / SCALE_MWH;
P_max_s, P_max_w

(95897.82943000001, 11158.008)

In [8]:
Availability_matrix = CSV.read("Availability.csv", DataFrame);
println(size(Availability_matrix))

(8760, 5)


In [9]:
demand = CSV.read("demand.csv", DataFrame) ./ SCALE_MWH;
println(size(demand))
first(demand, 5)

(8760, 4)


Row,Commercial,Industrial,Residential,Transportation
,Float64,Float64,Float64,Float64
1,177.72,109.564,169.055,55.3949
2,174.776,108.33,164.425,37.6361
3,173.069,102.035,165.18,33.1084
4,171.107,96.909,168.945,29.3248
5,171.561,94.8884,173.68,23.6573


In [10]:
ramp_limits = CSV.read("ramp_limits.csv", DataFrame)

Row,Ramp_limit
,Float64
1,1.0
2,0.1
3,0.025
4,0.03
5,1.0
6,1.0


In [11]:
curtailment_costs = [0, 2, 0, 0, 0, 0] ./ SCALE_DOLLAR

6-element Vector{Float64}:
 0.0
 0.002
 0.0
 0.0
 0.0
 0.0

Test without storage first

In [12]:
hrs = 4000
hre = 5000
sel_hrs = hrs:hre
println(sel_hrs)
techs = 5
n_eu = 4
# pad year by 1 day
add_h = 24

println("Cost")
C_FC = fixed_cost[1:techs,1]
C_FOM = fixed_cost[1:techs,2]
E_CO2 = co2_intensity[1:techs,1]
C_VOM = variable_cost[1:techs,1]
C_ramp = variable_cost[1:techs,2]
C_start = variable_cost[1:techs,3]
C_curt = curtailment_costs[1:techs]

println("Demand")
D_b = demand[sel_hrs, 1:4] # Baseline demand with no flex
println(size(D_b))
D_b = vcat(reverse(reverse(demand)[1:add_h, 1:n_eu]), D_b, D_b[1:24, 1:n_eu])
println(size(D_b))
total_demand = sum(eachcol(D_b))
X_D = total_demand

A_r = ramp_limits[1:techs, 1]
println("Availability")
A_s = Availability_matrix[sel_hrs,1:techs]
println("Try reverse")
A_s = vcat(reverse(reverse(Availability_matrix)[1:add_h, 1:techs]), A_s, A_s[1:24, 1:techs])
println(size(A_s))
# over time, opt each year with worst day for availability and highest demand?

4000:5000
Cost
Demand
(1001, 4)
(1049, 4)
Availability
Try reverse
(1049, 5)


# Constants

In [13]:
eta = 0.8; # Efficiency of the battery
CO2_price = 100; # price of the CO2 per ton
H = 4;

# Model

In [14]:
model = Model(Gurobi.Optimizer) # TODO: options? other optimizers?
set_optimizer_attribute(model, "NonConvex", 2)
# set_optimizer_attribute(model, "IterationLimit", 10000)
println("Model: initialized!\n")

n = size(C_FC, 1) # number of technologies
println("Number of technologies: ", n)
#T = size(D_b, 1) # hours in simulation
T = 50
println("Timespan: ", T)
time_ls = 1+add_h:T-add_h
vre_ls = 1:2
fossil_ls = 3:4
batt_ind = 5
# Non-negativity constraints
# The installed power capacity of the mix
@variable(model, P[1:n] >= 0)

# The hourly power generation of each technology
@variable(model, X_gen[1:T, 1:n] >= 0); # hourly production/use for each technology

# Curtailment (only renewables?)
@variable(model, X_cur[1:T, 1:n] >= 0); # hourly production/use for each technology

###### CONSTRAINTS ######

# Demand constraint - meet demand per hour
println("Demand constraint")
@constraint(model, Con_dh[t = 1:T], sum(X_gen[t, i] for i in 1:n) >= X_D[t]); # TODO: add curtailment
# Meet total demand
# @constraint(model, Con_d, sum(X_gen[t, i] for i in 1:n for t in 1:T) >= sum(total_demand));

# Installed capacity renewables < max capacity
println("Renewable maximum installed capacity constraint")
@constraint(model, Con_sgen, P[1] <= P_max_s);
@constraint(model, Con_wgen, P[2] <= P_max_w);

# Storage variable
println("Storage variable")
@variable(model, 0<= X_soc[1:T] <=1)
@variable(model, X_ch[1:T]>=0)

# Cannot produce more than the installed power
# println("Installed power constraint")
# @constraint(model, Con_p[i = 1:n, t = 1:T], X_gen[t, i] <= P[i])

# For each tech, at each hour, usage cannot exceed available capacity
println("Availability & installed power constraint")
@constraint(model, [i = 1:n, t = 1:T], X_gen[t, i] <= P[i]*A_s[t, i])
# Renewables produce maximum possible capacity
# @constraint(model, [i = vre_ls, t = 1:T], X_gen[t, i] == P[i]*A_s[t, i]) 

println("Curtailment definition")
@constraint(model, [t = 1:T], sum(X_cur[t, i] for i in 1:n) == sum(X_gen[t, i] for i in 1:n) - X_D[t]) 
@constraint(model, [i = 1:n, t = 1:T], X_cur[t, i] <= X_gen[t, i])

# Cannot produce less than the minimum stable generation: X_gen must be zero or min to max
# min_usage = [0, 0.1, 0, 0] #TODO - cant be more than ramp
# @constraint(model, [i = 1:n, t = 1:T], X_gen[t, i] >= P[i]*min_usage[i])

# For each tech, at each hour, change in production cannot exceed ramp rate (TODO might need to convert this to time)
println("Ramp constraint")
# Dummy rate of change variable
# @variable(model, X_roc[time_ls, 1:n])
# @constraint(model, [i = 1:4, t = time_ls], X_roc[t, i] == X_gen[t, i]-X_gen[limit_bound_max(t+1, T), i])

# Auxiliary variables to linearize the rate of change constraint
@constraint(model, Con_rdwn[i = 1:n, t = time_ls], (X_gen[t+1, i] - X_gen[t, i]) >= -A_r[i]*X_gen[t, i]) 
@constraint(model, Con_rup[i = 1:n, t = time_ls], (X_gen[t+1, i] - X_gen[t, i]) <= A_r[i]*X_gen[t, i]);

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-07
Set parameter NonConvex to value 2
Model: initialized!

Number of technologies: 5
Timespan: 50
Demand constraint
Renewable maximum installed capacity constraint
Storage variable
Availability & installed power constraint
Curtailment definition
Ramp constraint


## Objective

In [15]:
# # Add objective function: minimize the total costs TODO: discount rates
# function start_cost(x_0, x_1, i)
#     if x_0 == 0
#         C_start[i]
#     else
#         0
#     end
# end

#TODO RAMP AND STARTUP COSTS

@objective(model, Min, 
    (
        sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:n) +
        sum(sum((C_VOM[i] + CO2_price * E_CO2[i])* X_gen[t, i] for i in 1:n) for t in time_ls) +
        sum(sum(C_curt[i] * X_cur[t, i] for i in 1:n) for t in time_ls)
));

# Storage model

Variable to indicate if $x_{g,t}<x_{d,t}$

In [16]:
M = 10000000


# Need to put this conditions above
@variable(model, Z_X_gen_greater_X_D[1:T], Bin)

@constraint(model, Z_X_gen_greater_X_D_1[t in 1:T], sum(X_gen[t,i] for i in 1:4) - X_D[t] <= M*Z_X_gen_greater_X_D[t])
@constraint(model, Z_X_gen_greater_X_D_2[t in 1:T], -sum(X_gen[t,i] for i in 1:4) + X_D[t] <= M*(1-Z_X_gen_greater_X_D[t]))



50-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 Z_X_gen_greater_X_D_2[1] : -X_gen[1,1] - X_gen[1,2] - X_gen[1,3] - X_gen[1,4] + 10000000 Z_X_gen_greater_X_D[1] ≤ 9.99939801365229e6
 Z_X_gen_greater_X_D_2[2] : -X_gen[2,1] - X_gen[2,2] - X_gen[2,3] - X_gen[2,4] + 10000000 Z_X_gen_greater_X_D[2] ≤ 9.99942780791255e6
 Z_X_gen_greater_X_D_2[3] : -X_gen[3,1] - X_gen[3,2] - X_gen[3,3] - X_gen[3,4] + 10000000 Z_X_gen_greater_X_D[3] ≤ 9.99943484100643e6
 Z_X_gen_greater_X_D_2[4] : -X_gen[4,1] - X_gen[4,2] - X_gen[4,3] - X_gen[4,4] + 10000000 Z_X_gen_greater_X_D[4] ≤ 9.99943572723534e6
 Z_X_gen_greater_X_D_2[5] : -X_gen[5,1] - X_gen[5,2] - X_gen[5,3] - X_gen[5,4] + 10000000 Z_X_gen_greater_X_D[5] ≤ 9.99943222237599e6
 Z_X_gen_greater_X_D_2[6] : -X_gen[6,1] - X_gen[6,2] - X_gen[6,3] - X_gen[6,4] + 10000000 Z_X_gen_greater_X_D[6] ≤ 9.99941208626585e6
 Z_X_gen_greater_X_D_2[7

Create the variable X_min = min(X_1, X_2, X_3)

In [17]:
# X_1, X_2, X_3 = [eta*(sum(X_gen[t,i] for i in Power_generation_ls)- X_D[t]) , eta*P3], (1- X_soc[t])*P[5]* H]
@variable(model, z1[t in 1:T], Bin)
@variable(model, z2[t in 1:T], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
println("Z_1 expression")
@constraint(model, z_1_1[t in 1:T], eta*P[5] - eta*(sum(X_gen[t,i] for i in 1:4)- X_D[t])   <=  M*z1[t])
@constraint(model, z_1_2[t in 1:T], -eta*P[5] + eta*(sum(X_gen[t,i] for i in 1:4)- X_D[t]) <= M*(1-z1[t]))


#Minimum between X_1 and X_2
@variable(model, min_X_1_X_2[1:T])

println("Min X_1, X_2 expression")
@constraint(model, X_1_X_2_lower_1[t in 1:T], min_X_1_X_2[t] <= eta*(sum(X_gen[t,i] for i in 1:4)- X_D[t]))
@constraint(model, X_1_X_2_lower_2[t in 1:T], min_X_1_X_2[t] <= eta*P[5])
@constraint(model, X_1_X_2_higher_1[t in 1:T], min_X_1_X_2[t] >= eta*(sum(X_gen[t,i] for i in 1:4)- X_D[t]) - M*(1-z1[t]))
@constraint(model, X_1_X_2_higher_2[t in 1:T], min_X_1_X_2[t] >= eta*P[5] - M*z1[t])


# z_2 equal 1 if min(X_1, X_2) is is greater than (1- x_soc[t])*h*E_max
println(" Take the min of min(X_1, X_2) and X_3")
@constraint(model, z_2_1[t in 1:T], (1- X_soc[t])* P[5] * H - min_X_1_X_2[t] <=  M*z2[t])
@constraint(model, z_2_2[t in 1:T], - (1- X_soc[t]) * P[5] * H + min_X_1_X_2[t] <= M*(1-z2[t]))


# Constraint for the variable X_ch
println(" Take the overall min")
@variable(model, min_X_1_X_2_X_3[1:T])

@constraint(model, X_1_X_2_X_3_lower_1[t in 1:T], min_X_1_X_2_X_3[t] <= min_X_1_X_2[t])
@constraint(model, X_1_X_2_X_3_lower_2[t in 1:T], min_X_1_X_2_X_3[t] <= (1- X_soc[t])*P[3]* H)
@constraint(model, X_1_X_2_X_3_higher_1[t in 1:T], min_X_1_X_2_X_3[t] >= min_X_1_X_2[t] - M*(1-z2[t]))
@constraint(model, X_1_X_2_X_3_higher_2[t in 1:T], min_X_1_X_2_X_3[t] >= (1- X_soc[t])*P[3]* H - M*z2[t]);



Z_1 expression
Min X_1, X_2 expression
 Take the min of min(X_1, X_2) and X_3
 Take the overall min


Expression of $X_{CH}$

In [18]:
#Expression of X_ch
@constraint(model, X_ch_expression[t in 1:T], X_ch[t]  == Z_X_gen_greater_X_D[t]*min_X_1_X_2_X_3[t]);

Expression of min of [X_gen[t]- X_D[t],  P[5], X_soc[t]*P3]* H]

In [19]:
@variable(model, y2[1:T], Bin)

# z_2 equal 1 if min(X_1, X_2) is is greater than (1- x_soc[t])*h*E_max
println(" Take the min of min(X_1, X_2) and X_3")
@constraint(model, y_2_1[t in 1:T], X_soc[t]* P[5] * H - P[5] <=  M*y2[t])
@constraint(model, y_2_2[t in 1:T], - X_soc[t] * P[5] * H + P[5] <= M*(1-y2[t]))


# Constraint for the variable X_ch
println(" Take the overall min")
@variable(model, min_extra_prod_P5_soc[1:T])

@constraint(model, min_extra_prod_P5_soc_lower_1[t in 1:T], min_extra_prod_P5_soc[t] <= P[5])
@constraint(model, min_extra_prod_P5_soc_lower_2[t in 1:T], min_extra_prod_P5_soc[t] <= X_soc[t]*P[5]* H)
@constraint(model, min_extra_prod_P5_soc_higher_1[t in 1:T], min_extra_prod_P5_soc[t] >= P[5] - M*(1-y2[t]))
@constraint(model, min_extra_prod_P5_soc_higher_2[t in 1:T], min_extra_prod_P5_soc[t] >= X_soc[t]*P[5]* H - M*y2[t]);



 Take the min of min(X_1, X_2) and X_3
 Take the overall min


Expression of $X_{dch}$

In [20]:
@variable(model, batt_dispatch[1:T], Bin)

#Expression of X_dch
# issue with this constraint
@constraint(model, X_dch_expression[t in 1:T], X_gen[t,5]  <= batt_dispatch[t]*min_extra_prod_P5_soc[t]);



Expression of state of charge

In [21]:
# First hour need to be equal to one
# Start with a charged battery
@constraint(model, SOC_t_1, X_soc[1] == 1)

#
@constraint(model, SOC_t[t in 2:T], (X_soc[t]-X_soc[t-1])*H*P[5] == (X_ch[t-1]- X_gen[t-1,5]));


In [22]:
#Optimize
optimize!(model)

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1223 rows, 1005 columns and 4173 nonzeros
Model fingerprint: 0x6dc14cde
Model has 549 quadratic constraints
Variable types: 755 continuous, 250 integer (250 binary)
Coefficient statistics:
  Matrix range     [2e-06, 1e+07]
  QMatrix range    [1e+00, 4e+00]
  QLMatrix range   [1e+00, 1e+07]
  Objective range  [2e-03, 1e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
  QRHS range       [1e+07, 1e+07]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve added 257 rows and 61 columns
Presolve time: 0.01s
Presolved: 2462 rows, 1215 columns, 7116 nonzeros
Presolved model has 244 bilinear constraint(s)
Variable types: 965 continuous, 250 integer (250 binary)

Root relaxation: objective 4.2679

## Results

In [23]:
value.(P)

5-element Vector{Float64}:
  11.867911158110132
   0.0
 687.4607611045011
   0.0
 973.1786174766685

In [24]:
X_gen_matrix = value.(X_gen);
    X_gen_matrix = transpose(X_gen_matrix)
size(X_gen_matrix)

(5, 50)

In [25]:
X_cur_matrix = value.(X_cur);
    X_cur_matrix = transpose(X_cur_matrix)
size(X_cur_matrix)

(5, 50)

In [26]:
using Plots
println(T)
@userplot StackedArea

# a simple "recipe" for Plots.jl to get stacked area plots
# usage: stackedarea(xvector, datamatrix, plotsoptions)
@recipe function f(pc::StackedArea)
    x, y = pc.args
    n = length(x)
    y = cumsum(y, dims=2)
    seriestype := :shape

    # create a filled polygon for each item
    for c=1:size(y,2)
        sx = vcat(x, reverse(x))
        sy = vcat(y[:,c], c==1 ? zeros(n) : reverse(y[:,c-1]))
        @series (sx, sy)
    end
end

a = X_gen_matrix[1, time_ls]
b = X_gen_matrix[2, time_ls]
c = X_gen_matrix[3, time_ls]
d = X_gen_matrix[4, time_ls]
e = X_gen_matrix[5, time_ls]
a_c = X_cur_matrix[1, time_ls] * -1
b_c = X_cur_matrix[2, time_ls] * -1
c_c = X_cur_matrix[3, time_ls] * -1
d_c = X_cur_matrix[4, time_ls] * -1
x = time_ls


time_ls = 1:T
plotly()
stackedarea(time_ls, [d c b a e], labels=["Coal" "CCGT" "Wind" "Solar" "Battery"], lw=0)
stackedarea!(time_ls, [d_c c_c b_c a_c], labels=["Coal, curtail" "CCGT, curtial" "Wind, curtail" "Solar, curtail"], lw=0)
plot!(time_ls, total_demand[time_ls], label="Demand", lc=:black, lw=1, ls=:dot)
# plot!(x, value.(X_D)[:], label="Demand, shifted", lc=:black, lw=2)
plot!(size=(800,400))

50


In [32]:
using Plots


@userplot StackedArea

# a simple "recipe" for Plots.jl to get stacked area plots
# usage: stackedarea(xvector, datamatrix, plotsoptions)
@recipe function f(pc::StackedArea)
    x, y = pc.args
    n = length(x)
    y = cumsum(y, dims=2)
    seriestype := :shape

    # create a filled polygon for each item
    for c=1:size(y,2)
        sx = vcat(x, reverse(x))
        sy = vcat(y[:,c], c==1 ? zeros(n) : reverse(y[:,c-1]))
        @series (sx, sy)
    end
end

a = X_gen_matrix[1,:]
b = X_gen_matrix[2,:]
c = X_gen_matrix[5,:]
sNames = ["Solar","Wind","Battery"]
nb_hours_ls = 1:T
println(nb_hours_ls)
x = nb_hours_ls

plotly()
stackedarea(x, [a b c], labels=reshape(sNames, (1,3)))
plot!(x, X_D, label="Demand", lc=:black, lw=2, ls=:dot)
#genplot(x, X_D[:])

1:50


In [44]:
# Compare availability of solar to wind
plotly()
plot(x, value.(A_s)[time_ls, 1]*P_max_s, label="Solar", lc=:black, lw=1)
plot!(x, b, label="Solar Prod", lc=:blue, lw=1)
plot!(x, value.(A_s)[time_ls, 2]*P_max_w, label="Wind", lc=:black, lw=1, ls=:dash)
plot!(size=(1000,400))

In [29]:
plotly()
ls_X_SOC = []
for elem in value.(X_soc)
    #print(elem)
    ls_X_SOC = [ls_X_SOC; elem]
end
println(ls_X_SOC)


Any[1.0, 1.000000000008005, 1.0000000000106197, 1.0000000000106197, 1.0000000000106197, 1.0000000000106197, 1.0000000000106197, 1.000000000008005, 1.000000000008005, 1.0000000000106197, 1.000000000000789, 1.0, 0.9980080196979654, 0.9929739179067881, 0.989094222588156, 0.9857430987870075, 0.9792937543357305, 0.9610122718017371, 0.9299356683426583, 0.8886977407172171, 0.8487291999163233, 0.8181705710514365, 0.8040021444355057, 0.8041814425499825, 0.822730897927729, 0.5876939293564916, 0.3411410892244057, 0.09114108922440571, 0.03869532069291622, 0.01283917638857597, 0.0, 0.0, 0.013418771038763109, 0.037964730569073964, 0.06784867389468424, 0.09155110538295846, 0.1334082581369292, 0.17753923059140975, 0.17753923059140975, 0.17753923059140975, 0.21579110970834145, 0.24446172740560926, 0.26064436021122345, 0.26064436021122345, 0.24860693549265708, 0.22335160787038072, 0.1861944533822617, 0.14466202373587392, 0.09968101271846219, 0.05115010608872781]


In [30]:
plot(1:T, ls_X_SOC, label="X min", lc=:red, lw=2, ls=:dot)
plot!(size=(800,400))

In [31]:
plot(1:T, value.(X_gen[:,5]), label="X min", lc=:red, lw=2, ls=:dot)
plot!(size=(800,400))